In [ ]:
# Beautiful Soup

## Introdução

Nesta aula veremos como utilizar na prática a biblioteca BeautifulSoup, muito útil para realizar web scraping em sites que não utilizam conteúdos dinâmicos.

Este laboratório tem como base a própria documentação da bilioteca, que pode ser encontrada em:
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

## Preâmbulo

Antes de rodar o código acima, é necessário instalar a biblioteca Beautiful Soup no seu ambiente de desenvolvimento Python.

É interessante criar um ambiente vitual de desenvolvimento antes de instalar a biblioteca. Neste curso, usaremos o conda para criar e gerenciar nossos ambientes virtuais.

Seguem os comandos para criar um abiente virtual e instalar a biblioteca no Python:

```
conda create --name ambiente-cpa-p3 python 3.12

conda activate ambiente-cpa-p3

conda install bs4 html5lib
```

---

## Obtendo páginas na Web

In [ ]:
# Essa função é utilizada para recuperar o html de uma página web
from urllib.request import urlopen

In [ ]:
# Abrindo uma página
html = urlopen('http://www.pythonscraping.com/pages/page1.html')
site = html.read()
print(site)

## Tratamento de Erros

Ao abrir uma url usano a função ``urlopen`` temos dois problemas principais que podem ocorrer:
- A página pode não estar no servidor
- O servidor não existe / não foi encontrado
No primeiro caso o servidor retornará um erro do tipo 404 - Page not found ou 500 - Internal server error. Em ambos os casos o python lança uma exceção do tipo ``HTTPError``. No segundo caso o erro que ocorre é ``URLError``.

Essas exceções devem sempre ser tratadas em nosso código, evitando erros desnecessários

In [ ]:
from urllib.error import HTTPError, URLError

def get_pagina(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        print("Houve um erro na obtenção da página! ", e)
        return None
    except URLError as e:
        print("Ocorreu um erro no servidor!", e)
        return None
    else:
        print("Consegui abrir a página")
        return html.read()
    

In [ ]:
pagina = get_pagina('http://www.google.com.br/lucas')
if pagina is not None:
    print(pagina)


In [ ]:
pagina = get_pagina('http://www.joogle.com.br/')
if pagina is not None:
    print(pagina)

In [ ]:
pagina = get_pagina('https://www.google.com.br')
if pagina is not None:
    print(pagina)

---

## Arquivo ``robots.txt``

In [ ]:
import urllib.robotparser

In [ ]:
# Lê o arquivo robots.txt
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://g1.globo.com/robots.txt')
rp.read()

In [ ]:
rrate = rp.can_fetch("*", "https://g1.globo.com/")
print(rrate)

In [ ]:
rrate = rp.can_fetch("GPTBot", "https://g1.globo.com/")
print(rrate)

In [ ]:
rrate = rp.can_fetch("*", "https://g1.globo.com/jornalismo/g1/")
print(rrate)

---

## Basicão do BS

In [ ]:
# Importando a classe principal da biblioteca BeautifulSoup
from bs4 import BeautifulSoup

In [ ]:
# Cria o objeto BeautifulSoup
bs = BeautifulSoup(site, 'html.parser')
print(bs.prettify())

In [ ]:
print(bs.h1)

In [ ]:
print(bs.h1.text)

Note que essa forma de indexar os elementos do HTML só permite retornar a **primeira ocorrência** de cada um. Por exemplo, se houver duas tagas `h1` em uma página somente a primeira poderá ser obtida dessa forma.

### Exercício
Recupere o conteúdo da tag `div`

---

Abaixo criaremos uma variável com o HTML que utilizaremos como exemplo neste laboratório. É um trecho de "Alice no País das Maravilhas" formatado em HTML.

In [ ]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

In [ ]:
soup = BeautifulSoup(html_doc)
print(soup.prettify())

## Acesso aos atributos e Movimentação

Podemos acessar os atributos de forma semelhante com a qual acessamos os valores de um dicionário.

In [ ]:
soup.body.a['href']

### Exercício
Escreva o código que acessa o conteúdo do atributo class da primeira tag "p" que está dentro de body
A saída deve ser: `['title']`


Para acessar todos os filhos de uma tag, podemos utilizar o método ```.contents``` ou o gerador de listas ```.children```

In [ ]:
tag_head = soup.head
tag_head

In [ ]:
tag_head.contents

Em alguns casos podemos querer também os “filhos dos filhos”, nesse caso podemos utilizar o método ```.descendants```

In [ ]:
for filho in tag_head.descendants:
    print(filho)

Quando a tag possui apenas uma ```NavigableString``` como filho, podemos acessar pelo ```.string```, caso possua mais de um podemos acessar via ```.strings``` e ```.stripped_strings```


In [ ]:
tag_titulo = soup.head.title
tag_titulo

In [ ]:
tag_titulo.string

In [ ]:
print(type(tag_titulo.string))
titulo = str(tag_titulo.string)
print(titulo, type(titulo))

Podemos visitar também tags irmãs acessando os métodos ```.next_siblings``` e ```.previous_siblings```

In [ ]:
link = soup.a
print(link)

In [ ]:
link.next_sibling

In [ ]:
for irmao in link.next_siblings:
    print("[", irmao, "]")

## Funções de Busca

As funções de busca tem como objetivo encontrar elementos dentro das páginas web. Existem duas funções de busca: `find` e `find_all`

### Busca por string

In [ ]:
soup.find('a')

In [ ]:
soup.find_all('a')

### Exercício
Qual a diferença entre as funções find e find_all?

### Busca por Regex

In [ ]:
import re 
for tag in soup.find_all(re.compile("^b")):
    print(tag.name)

### Busca por lista
Match com um elemento de uma lista

In [ ]:
soup.find_all(["a", "b"])

### True

In [ ]:
for tag in soup.find_all(True):
    print(tag.name)

### Função

Match com elementos que retornam True quando passados para a função

In [ ]:
def has_class_but_no_id(tag):
   return tag.has_attr('class') and not tag.has_attr('id')

for tag in soup.find_all(has_class_but_no_id):
   print(tag)
   print()

In [ ]:
from bs4 import NavigableString
def surrounded_by_strings(tag):
    return (isinstance(tag.next_element, NavigableString)
            and isinstance(tag.previous_element, NavigableString))

for tag in soup.find_all(surrounded_by_strings):
    print(tag.name)   

### Parâmetros da função find_all

```
find_all(name, attrs, recursive, string, limit, **kwargs)
```

```name```: para filtrar apenas tags com o nome específico

```attrs```: utilizado para realizar filtros de atributos

```recursive```: pesquisar apenas na tag ou em seus descendentes

```string```: pesquisar pelo conteúdo/string das tags

```limit```: limita o número de retornos do ```find_all```

```**kwargs```: Todos os parâmetros nomeados não conhecidos são convertidos para filtros de atributos

In [ ]:
# Parâmetro nome
soup.find_all("title")

In [ ]:
# Parâmetro attrs
soup.find_all(attrs={"class": "sister"})

In [ ]:
# Atributo recursive
soup.html.find_all("title")

In [ ]:
# Esse código só busca nos filhos diretos da tag html
soup.html.find_all("title", recursive=False)

In [ ]:
# Atributo string
soup.find_all(string="Elsie")

In [ ]:
soup.find_all(string=["Tillie", "Lacie", "Elsie"])

In [ ]:
import re
soup.find_all(string=re.compile("Dormouse"))

In [ ]:
def is_the_only_string_within_a_tag(s):
    """Retorna True se a string for o filho único da tag pai."""
    return (s == s.parent.string)

soup.find_all(string=is_the_only_string_within_a_tag)

In [ ]:
# Parâmetro limit
soup.find_all("a", limit=2)

In [ ]:
# Parâmetro **kwargs
soup.find_all(id="link2")

In [ ]:
soup.find_all(href=re.compile("elsie"))

In [ ]:
soup.find_all(class_="sister")

### Exercício

Usando ```find_all``` selecione:
1. Todas as tags da classe story
2. Tag com id link2 e as suas irmãs subsequentes 


In [ ]:
# exercicio 1
soup.find_all(attrs={"class": "story"})

In [ ]:
# exercicio 2


## Seletores CSS

Seletores são padrões textuais que casam com algum elemento de uma árvore (de um arquivo XML ou HTML). 

Podemos ter os seguintes tipos de seletores:

* **Seletores Simples**: Por tipo, universal, por id, por classe, por atributo ou pseudo-classe;
* **Seletores Compostos**: É uma sequência de seletores simples (por exemplo por tipo e por classe);
* **Lista de Seletores**: É formada por uma lista de seletores separadas por vírgula
* **Seletores Complexos**: É formado por múltiplos seletores simples ou compostos junto de combinadores.


In [ ]:
# Seletor por tipo
soup.select("title")

In [ ]:
# Seletor Universal
soup.select("*")

In [ ]:
# Seletor por atributo
soup.select("[id]")

In [ ]:
# Seletor de atributo por substring
soup.select("[class^=s]")

In [ ]:
# Seletor de atributo por classe
soup.select(".sister")

In [ ]:
# Seletor de Id
soup.select("#link2")

In [ ]:
# Seletores Compostos
soup.select("a[href$=tillie]")

### Seletores com Combinadores

In [ ]:
# Combinador Espaço ' '
soup.select("body a")

In [ ]:
# Combinador Filho >
soup.select("head > title")

In [ ]:
# Combinador Irmão Subsequente ~
soup.select("#link1 ~ .sister")

### Exercício

Crie um seletor CSS para cada item abaixo.
1. Todas as tags b que estão dentro de um p
2. Todas as tags da classe story
3. Tag com id link2 e as suas irmãs subsequentes 